In [1]:
# The following two lines are not needed if you have topologicpy already pip installed.
import sys
sys.path.append("C:/Users/wassimj/Documents/GitHub/")

In [2]:
from topologicpy.Vertex import Vertex
from topologicpy.Edge import Edge
from topologicpy.Wire import Wire
from topologicpy.Face import Face
from topologicpy.Shell import Shell
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex
from topologicpy.Cluster import Cluster
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from topologicpy.Graph import Graph
from topologicpy.Plotly import Plotly
from topologicpy.Color import Color
from topologicpy.Vector import Vector
from topologicpy.Helper import Helper
from topologicpy.Graph import Graph
from topologicpy.DGL import DGL
import topologicpy
import topologic
import math
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interact_manual
print("Done importing libraries")

Done importing libraries


In [13]:
# nt: int - Number of towers
# nf: int - number of floors
# columns: bool - make columns yes or no
# plinth: bool - make plinth yes or no
# core: int - number of cores per building can be 1 or 2 max
# basement: bool - True of False include a basement
def getSelectors(cells, keys, values):
    if not cells:
        return None
    selectors = []
    for cell in cells:
        s = Topology.InternalVertex(cell)
        d = Dictionary.ByKeysValues(keys, values)
        s = Topology.SetDictionary(s, d)
        selectors.append(s)
    return selectors

def makeBuilding(nt=1,nf=10, core=1, basement=False, plinth=False, columns=False, opacity=0.1,
                path="C:/Users/wassimj/Desktop/csv-trained-model.pt"):
    Topology.Cleanup()
    ground = CellComplex.Prism(width=54, length=54, uSides=5, vSides=5, height=6, wSides=1, placement="bottom")
    offset = 6
    if basement:
        offset = offset-6
        nf = nf+1
    bldg_origins = []
    if nt == 1:
        bldg_origins.append(Vertex.ByCoordinates(0,0,offset))
    elif nt == 2:
        bldg_origins.append(Vertex.ByCoordinates(-16,0,offset))
        bldg_origins.append(Vertex.ByCoordinates(16,0,offset))
    elif nt == 3:
        bldg_origins.append(Vertex.ByCoordinates(-16,-16,offset))
        bldg_origins.append(Vertex.ByCoordinates(-16,16,offset))
        bldg_origins.append(Vertex.ByCoordinates(16,16,offset))
    else:
        bldg_origins.append(Vertex.ByCoordinates(-16,-16,offset))
        bldg_origins.append(Vertex.ByCoordinates(-16,16,offset))
        bldg_origins.append(Vertex.ByCoordinates(16,16,offset))
        bldg_origins.append(Vertex.ByCoordinates(16,-16,offset))
    core_origins = []
    if core == 1:
        for bldg_origin in bldg_origins:
            core_origins.append(bldg_origin)
            core_length=3
    else:
        for bldg_origin in bldg_origins:
            v1 = Topology.Translate(bldg_origin, -4.5, 0, 0)
            v2 = Topology.Translate(bldg_origin, 4.5, 0, 0)
            core_origins += [v1,v2]
            core_length=5
    buildings = []
    for bldg_origin in bldg_origins:
        buildings.append(CellComplex.Prism(origin=bldg_origin, width=12, length=12, height=nf*3,
                                            uSides=2, vSides=2, wSides=int(nf/2), placement="bottom"))
    bldg_clus = Cluster.ByTopologies(buildings)
    cores = []
    for core_origin in core_origins:
        cores.append(Cell.Prism(origin=core_origin, width=3, length=core_length, height=nf*3+3,
                                            uSides=1, vSides=1, wSides=1, placement="bottom"))
    core_clus = Topology.SelfMerge(Cluster.ByTopologies(cores))
    plinths = []
    plinth_clus = None
    if plinth:
        for bldg_origin in bldg_origins:
            plinth_origin = Vertex.ByCoordinates(Vertex.X(bldg_origin), Vertex.Y(bldg_origin), 6)
            plinths.append(Cell.Prism(origin=plinth_origin, width=14, length=14, height=3, placement="bottom"))
        plinth_clus = Cluster.ByTopologies(plinths)
    cols = []
    cols_clus = None
    subtract_vols = []
    subtract_clus = None
    if columns:
        for bldg_origin in bldg_origins:
            subtract_vols.append(Cell.Prism(origin=bldg_origin, width=16, length=16, height=3, placement="bottom"))
            if plinth:
                z = 9
            else:
                z = 6
            v1= Vertex.ByCoordinates(Vertex.X(bldg_origin), Vertex.Y(bldg_origin), z)
            subtract_vols.append(Cell.Prism(origin=v1, width=16, length=16, height=3, placement="bottom"))
            v1 = Vertex.ByCoordinates(Vertex.X(bldg_origin)-3, Vertex.Y(bldg_origin)-3, z)
            v2 = Vertex.ByCoordinates(Vertex.X(bldg_origin)+3, Vertex.Y(bldg_origin)-3, z)
            v3 = Vertex.ByCoordinates(Vertex.X(bldg_origin)+3, Vertex.Y(bldg_origin)+3, z)
            v4 = Vertex.ByCoordinates(Vertex.X(bldg_origin)-3, Vertex.Y(bldg_origin)+3, z)
            col_origins = [v1,v2,v3,v4]
            for col_origin in col_origins:
                cols.append(Cell.Prism(origin=col_origin, width=1, length=1, height=3, placement="bottom"))
        cols_clus = Cluster.ByTopologies(cols)
        subtract_clus = Cluster.ByTopologies(subtract_vols)
        bldg_clus = Topology.Difference(bldg_clus, subtract_clus)
    ground = Topology.Difference(ground, bldg_clus)
    bldg_clus = Topology.Difference(bldg_clus, core_clus)
    if plinth:
        bldg_clus = Topology.Difference(bldg_clus, plinth_clus)
        plinth_clus = Topology.Difference(plinth_clus, core_clus)
    ground_data = Plotly.DataByTopology(ground, faceColor="purple")
    if plinth:
        plinth_data = Plotly.DataByTopology(plinth_clus, faceColor="lightgrey")
    else:
        plinth_data = []
    if columns:
        columns_data = Plotly.DataByTopology(cols_clus, faceColor="grey")
    else:
        columns_data = []
    bldg_data = Plotly.DataByTopology(bldg_clus, faceColor="green")
    core_data = Plotly.DataByTopology(core_clus, faceColor="yellow")
    all_data = ground_data+plinth_data+columns_data+bldg_data+core_data
    fig = Plotly.FigureByData(all_data)
    Plotly.Show(fig)
    # Get cells and create selectors
    selectors = []
    cells = []
    grd_cells = Topology.Cells(ground)
    cells.append(grd_cells)
    selectors.append(getSelectors(grd_cells, ["label","title"], [0, "ground"]))
    if plinth:
        if isinstance(plinth_clus, topologic.Cell):
            plinth_cells = [plinth_clus]
        else:
            plinth_cells = Topology.Cells(plinth_clus)
        cells.append(plinth_cells)
        selectors.append(getSelectors(plinth_cells, ["label", "title"], [1, "plinth"]))
    if columns:
        column_cells = Topology.Cells(cols_clus)
        cells.append(column_cells)
        selectors.append(getSelectors(column_cells, ["label", "title"], [2, "column"]))
    bldg_cells = Topology.Cells(bldg_clus)
    cells.append(bldg_cells)
    selectors.append(getSelectors(bldg_cells, ["label", "title"], [3, "building"]))
    if isinstance(core_clus, topologic.Cell):
        core_cells = [core_clus]
    else:
        core_cells = Topology.Cells(core_clus)
    cells.append(core_cells)
    selectors.append(getSelectors(core_cells, ["label", "title"], [4, "core"]))
    selectors = Helper.Flatten(selectors)
    cells = Helper.Flatten(cells)
    everything = CellComplex.ByCells(cells)
    everything = Topology.TransferDictionariesBySelectors(everything, selectors, tranCells=True)
    g = Graph.ByTopology(everything)
    graph_data = Plotly.DataByGraph(g, vertexSize=4, vertexGroupKey="label", vertexLabelKey="title", vertexGroups=[0,1,2,3,4])
    bldg_data = Plotly.DataByTopology(everything, faceOpacity=opacity, edgeColor="lightgrey", vertexColor="lightgrey")
    fig = Plotly.FigureByData(graph_data+bldg_data)
    Plotly.Show(fig)
    graph_data = Plotly.DataByGraph(g, vertexSize=6, vertexGroupKey="label", vertexLabelKey="title", vertexGroups=[0,1,2,3,4])
    fig = Plotly.FigureByData(graph_data)
    Plotly.Show(fig)
    dgl_graph = DGL.GraphByTopologicGraph(g, bidirectional=True, key="label", categories=[0,1,2,3,4], node_attr_key='node_attr', tolerance=0.0001)
    dataset = DGL.DatasetByGraphs({"graphs":[dgl_graph], "labels":[0]})
    model = DGL.ModelLoad(path=path) # Change the path!
    dictionary = DGL.ModelClassify(model, dataset)
    predicted = dictionary['predictions']
    prediction = predicted[0]
    categories = ["Separation", "Separation with Plinth", "Adherence", "Adherence with Plinth", "Interlock"]
    print("PREDICTION:",prediction,"-",categories[prediction])
        

In [14]:
nt_w = widgets.IntSlider(min=1,max=4,step=1,value=1, description="# Towers", continuous_update=False)
nf_w = widgets.IntSlider(min=4,max=10,step=1,value=4, description="# Floors", continuous_update=False)
nc_w = widgets.IntSlider(min=1,max=2,step=1,value=1, description="# Cores", continuous_update=False)
b_w= widgets.Checkbox(
    value=False,
    description='Basement',
    disabled=False
)
p_w= widgets.Checkbox(
    value=False,
    description='Plinth',
    disabled=False
)
c_w= widgets.Checkbox(
    value=False,
    description='Columns',
    disabled=False
)
path_w = widgets.Text(
    placeholder="C:/Users/wassimj/Desktop/csv-trained-model.pt",
    value="C:/Users/wassimj/Desktop/csv-trained-model.pt",
    description='Model Path:',
    disabled=False
    )
o_w = widgets.FloatSlider(min=0,max=1.0,step=0.1,value=0.5, description="Opacity", continuous_update=False)
my_interact_manual = interact_manual.options(manual_name="Display Building")
_=my_interact_manual(makeBuilding,nt=nt_w, nf=nf_w, core=nc_w, basement=b_w, plinth=p_w, columns=c_w, opacity=o_w, path=path_w)

interactive(children=(IntSlider(value=1, continuous_update=False, description='# Towers', max=4, min=1), IntSl…